In [1]:
from bs4 import BeautifulSoup
import requests 
import re
import pandas as pd
import numpy as np

In [2]:
def get_title(soup):
    try:
        # Extract product title
        title = soup.find("span", attrs={"id":'productTitle'}).text.strip()
    except AttributeError:
        title = ""
    return title

def get_price(soup):
    try:
        # Extract product price
        price = soup.find("span", attrs={"class":'a-price a-text-price a-size-medium apexPriceToPay'}).find("span", attrs={"class": "a-offscreen"}).text
    except AttributeError:
        price = ""
    return price

def get_ratings(soup):
    try:
        # Extract product rating
        rating = soup.find("span", attrs={"class":'a-icon-alt'}).text
    except AttributeError:
        rating = ""
    return rating

In [4]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'}

    # The webpage URL
    URL = "https://www.amazon.com/s?k=bag&crid=3A06VPJI3SKQG&sprefix=bag%2Caps%2C322&ref=nb_sb_noss_1"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    # # # Store the links
    links_list = []

    # # # Loop for extracting links from Tag Objects
    # Regular expression to filter out unwanted links
    pattern = re.compile(r'https://aax-us-iad\.amazon\.com')

    # Loop for extracting links from Tag Objects
    for link in links:
        href = "https://www.amazon.com" + link.get('href')
        if not href.startswith("https://www.amazon.com") or pattern.search(href):
            
            continue

        links_list.append(href)

    d = {"title": [], "price": [], "rating": []}
    for link in links_list:
        new_webpage = requests.get(link, headers=HEADERS)
        print(f"Scraping: {link}")

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        title = get_title(new_soup)
        price = get_price(new_soup)
        rating = get_ratings(new_soup)

        # print(f"Title: {title}, Price: {price}, Rating: {rating}")

        d['title'].append(title)
        d['price'].append(price)
        d['rating'].append(rating)
    # # # Loop for extracting product details from each link 
    # for link in links_list:
    #     if not link.startswith("https://www.amazon.com"):
    #         print(f"Skipping invalid link: {link}")
    #         continue
    #         new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

    #         new_soup = BeautifulSoup(new_webpage.content, "html.parser")

    #         # Function calls to display all necessary product information
    #         d['title'].append(get_title(new_soup))
    #         d['price'].append(get_price(new_soup))
    #         d['rating'].append(get_rating(new_soup))
            
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

Scraping: https://www.amazon.com/Vintage-Corduroy-Crossbody-Shoulder-Handbags/dp/B0B2836G9Y/ref=sr_1_1?crid=3A06VPJI3SKQG&dib=eyJ2IjoiMSJ9.Lohftufpuy0YbPk0VDoSDaPpwmPtYKsHhKsL_7A84iZT85eW09C27OSat6N61bCLSiPj6Hn8XMYzEMN-bzPGmeTQn8kVbt389H7un_Q6nBYYClR1qeuxpEV2ZXB65cHXf4urdDSMnbE0hI6jaddpSfnWbeZVcAYna40PuxdI7pm1sR1DXU0F6N75FTaXJ4s1q_fwikMcKmVYaw_vxpcIcFnn6DZvoRtuAJ9Vc9wvOVc.s0DGXPXwQFzbEA4iuIB6buCSbAwoTv_tIAWnAdJSO4A&dib_tag=se&keywords=bag&qid=1705405241&sprefix=bag%2Caps%2C322&sr=8-1
Scraping: https://www.amazon.com/Duffel-Charging-Weekender-Overnight-Compartment/dp/B09J282WZD/ref=sr_1_2?crid=3A06VPJI3SKQG&dib=eyJ2IjoiMSJ9.Lohftufpuy0YbPk0VDoSDaPpwmPtYKsHhKsL_7A84iZT85eW09C27OSat6N61bCLSiPj6Hn8XMYzEMN-bzPGmeTQn8kVbt389H7un_Q6nBYYClR1qeuxpEV2ZXB65cHXf4urdDSMnbE0hI6jaddpSfnWbeZVcAYna40PuxdI7pm1sR1DXU0F6N75FTaXJ4s1q_fwikMcKmVYaw_vxpcIcFnn6DZvoRtuAJ9Vc9wvOVc.s0DGXPXwQFzbEA4iuIB6buCSbAwoTv_tIAWnAdJSO4A&dib_tag=se&keywords=bag&qid=1705405241&sprefix=bag%2Caps%2C322&sr=8-2
Scraping: https://w

In [5]:
amazon_df

,title,price,rating
0,Valleycomfy Vintage Casual Corduroy Tote Bags ...,$15.99,4.5 out of 5 stars
1,"ETRONIK Gym Bag for Women, Travel Duffel Bag w...",$36.99,4.6 out of 5 stars
2,"BAGSMART Women Tote Bag with Zipper, Laptop Tr...",$32.99,4.7 out of 5 stars
3,TOPDesign Personalized Initial Canvas Beach Ba...,$14.44,4.8 out of 5 stars
4,"AIYUENCICI Crochet Tote Bag, Fairy Hobo Bag fo...",$9.34,4.4 out of 5 stars
...,...,...,...
72,"The Tote Bag for Women, PU Leather Tote Bag, S...",$26.99,4.3 out of 5 stars
73,"MATEIN Travel Laptop Backpack, Business Anti T...",$29.99,4.7 out of 5 stars
74,HOLYLUCK Drawstring Backpack Bag Sport Gym Sac...,$9.88,4.7 out of 5 stars
75,"charlore The Leather Tote Bag For Women, Women...",$39.99,4.5 out of 5 stars
